In [2]:
!pip install torch

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [6]:
ngrams = [
  ([test_sentence[i-j-1] for j in range(CONTEXT_SIZE)], test_sentence[i]) for i in range(CONTEXT_SIZE, len(test_sentence))
]

In [7]:
print(ngrams[:5])

[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege'), (['besiege', 'shall'], 'thy'), (['thy', 'besiege'], 'brow,')]


In [8]:
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

In [15]:
class NGramLanguageModel(nn.Module):
  def __init__(self, vocab_size, embedding_dim, context_size):
    super(NGramLanguageModel, self).__init__()
    self.embeddings = nn.Embedding(vocab_size, embedding_dim)
    self.linear1 = nn.Linear(context_size * embedding_dim, 128)
    self.linear2 = nn.Linear(128, vocab_size)

  def forward(self, inputs):
    embeds = self.embeddings(inputs).view((1, -1))
    #print(len(embeds), embeds.shape)
    out = F.relu(self.linear1(embeds))
    out = self.linear2(out)
    log_probs = F.log_softmax(out, dim=1)
    return log_probs

losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModel(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
print(model)
optimizer = optim.SGD(model.parameters(), lr=0.001)


NGramLanguageModel(
  (embeddings): Embedding(97, 10)
  (linear1): Linear(in_features=20, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=97, bias=True)
)


In [19]:
print(model.embeddings.weight[word_to_ix["beauty"]])

tensor([ 0.3732,  0.7156, -2.0364, -1.3711,  0.5691,  0.0463, -1.2170,  0.4473,
         1.3707, -0.5040], grad_fn=<SelectBackward0>)


In [30]:
for epoch in range(1):
  total_loss = 0
  for context, target in ngrams:
    context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
    model.zero_grad()
    log_probs = model(context_idxs)
    loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
    loss.backward()

    if total_loss == 0:
      print(log_probs)
      print(f"({[w for w in context]} --> context_idxs: {context_idxs}")
      print(f"log_probs shape: {log_probs.shape}")
      print(f"target: {target} --> tensor: {torch.tensor([word_to_ix[target]], dtype=torch.long)}")
      print(f"loss:{loss}")

    optimizer.step()

    total_loss += loss.item()

  losses.append(total_loss)
#print(losses)

#print(model.embeddings.weight[word_to_ix["beauty"]])


tensor([[-4.6247, -4.4294, -4.7861, -4.8080, -4.3893, -4.5652, -4.4907, -5.0925,
         -4.5138, -4.6311, -4.5053, -4.5276, -4.4009, -4.6776, -4.7038, -4.4928,
         -4.5821, -4.5776, -4.4770, -4.5180, -4.7538, -4.6650, -4.7006, -4.3737,
         -4.5134, -4.5906, -4.5438, -4.8021, -4.6205, -4.3649, -4.5806, -4.6948,
         -4.7118, -4.3908, -4.6374, -4.3672, -4.3520, -4.3926, -4.5523, -4.6725,
         -4.3855, -4.2992, -4.3051, -4.6683, -4.2017, -4.8586, -4.5333, -4.6584,
         -4.2731, -4.6710, -4.7398, -4.6757, -4.5226, -4.5369, -4.3528, -4.5146,
         -4.6574, -4.6271, -4.4995, -4.5729, -4.8208, -4.3304, -4.4828, -4.7519,
         -4.4630, -4.7288, -4.5895, -4.6883, -4.6951, -4.4781, -4.2842, -4.4752,
         -4.5586, -4.4386, -4.5726, -4.4723, -4.3040, -4.7928, -4.6937, -4.8473,
         -4.5487, -4.8418, -4.8265, -4.4305, -4.8743, -4.7772, -4.9604, -4.6414,
         -4.7349, -4.7314, -4.7835, -4.3876, -4.9405, -4.6881, -4.6131, -4.6850,
         -4.5923]], grad_fn=